1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 
2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [5]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-12) + (1.0 - y) * np.log(1.0 - y_pred + 1e-12))
    return err
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [6]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [7]:
W = eval_model(X_st, y, iterations=100000, alpha=1e-1)

10000 [-11.27241705  -1.45342424  -2.38315559   9.49424167] 0.25238324894799624
20000 [-16.2427091   -1.86940882  -3.90475531  13.50901763] 0.2087442172523697
30000 [-20.12083462  -2.20557401  -5.06349228  16.65543794] 0.18225097916773386
40000 [-23.32991383  -2.48704062  -6.00960348  19.2582858 ] 0.16415855743791508
50000 [-26.07030122  -2.7286846   -6.81038328  21.47869092] 0.15099538645483848
60000 [-28.46500332  -2.94026656  -7.50582736  23.41651039] 0.1409638346257683
70000 [-30.59553066  -3.12847345  -8.12185755  25.13810328] 0.13303779902564522
80000 [-32.5184265   -3.29804548  -8.67615891  26.68956031] 0.12659210791726486
90000 [-34.27432938  -3.45244383  -9.1812758   28.10403169] 0.12122579824316748
100000 [-35.8933164   -3.59426199  -9.64639296  29.4060647 ] 0.11667060790985617


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [8]:
def calc_pred_proba(W, x):
    y_pred_proba = sigmoid(np.dot(W.T, x))
    return y_pred_proba

In [9]:
calc_pred_proba(W, X_st)

array([0.34705568, 0.00984648, 1.        , 0.06777527, 0.93622651,
       0.04716209, 1.        , 0.01783172, 0.58968863, 0.99999976])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [10]:
def calc_pred(W, X, t=0.5):    
    y_pred = sigmoid(np.dot(W.T, X)) >= t
    y_pred = y_pred.astype(np.int64)
    return y_pred

In [11]:
y_pred = calc_pred(W, X_st, t=0.5)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [12]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
ac = y - y_pred
accuracy = np.sum([ac == 0] ) / 10
accuracy

1.0

In [14]:
def matr(y, y_pred):
    TP=0
    TN=0
    FP=0
    FN=0
    for i in range(len(y)):
        if y[i] == 1:
            if y[i] == y_pred.T[i]:
                TP += 1
            else:
                FN += 1
        else:
            if y[i] != y_pred.T[i]:
                FP += 1
            else:
                TN += 1
    TP/=len(y)
    TN/=len(y)
    FP/=len(y)
    FN/=len(y)
    m  = [[TP, FP], [FN, TN]]
    return m, TP, FP, FN, TN
m, TP, FP, FN, TN = matr(y, y_pred)
m

[[0.5, 0.0], [0.0, 0.5]]

In [15]:
precision = TP / (TP + FP)
precision

1.0

In [16]:
recall = TP / (TP + FN)
recall

1.0

In [17]:
F1 = 2 * precision * recall / (precision + recall)
F1

1.0

6. Могла ли модель переобучиться? Почему?

Могла, из-за малой выборки и ручной подстройки порога для разбияния на категории. И при добавлении регуляризации переобучение должно также снизиться.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [18]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 *  np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [19]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W